In [1]:
import pandas as pd
# train = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/annealing/anneal.data")
# test = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/annealing/anneal.test")
train =pd.read_csv('anneal.data')
test = pd.read_csv('anneal.test')

In [2]:
columns = ["family", "product-type", "steel", "carbon", "hardness", "temper_rolling",
"condition", "formability", "strength", "non-ageing", "surface-finish", "surface-quality",
"enamelability", "bc", "bf", "bt", "bw/me", "bl", "m", "chrom", "phos", "cbond", "marvi",
"exptl", "ferro", "corr", "blue/bright/varn/clean", "lustre", "jurofm", "s", "p", "shape",
"thick", "width", "len", "oil", "bore", "packing", "classes"]

In [3]:
W = pd.concat([train, test], ignore_index=True)

In [ ]:
W.shape

In [ ]:
W.iloc[:,-3:]

In [75]:
import copy as cp

In [4]:
z = W.iloc[:,:-3]
# z.shape

In [5]:
z.columns = columns
X = z.iloc[:,:-1]
y = z.iloc[:,-1]

In [ ]:
X.columns

In [ ]:
y

#### [Remove columns with over 20% missing values](https://stackoverflow.com/questions/37921703/how-to-delete-columns-with-at-least-20-missing-values)

In [6]:
Xx = X.loc[:, X.isin(["?"]).sum()<len(X)*.20]
Xx.isin(['?']).sum(axis=0)

product-type     0
steel           86
carbon           0
hardness         0
strength         0
shape            0
thick            0
width            0
len              0
bore             0
dtype: int64

#### Add accidently deleted column (because it had over 20% missing data so got deleted now join again)

In [7]:
Xx['surface_quality'] = X['surface-quality']

/home/julia/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
Xx.columns

Index(['product-type', 'steel', 'carbon', 'hardness', 'strength', 'shape',
       'thick', 'width', 'len', 'bore', 'surface_quality'],
      dtype='object')

In [9]:
Xx.isin(['?']).sum(axis=0)

product-type         0
steel               86
carbon               0
hardness             0
strength             0
shape                0
thick                0
width                0
len                  0
bore                 0
surface_quality    217
dtype: int64

#### Determine how many classes are in respective groups/columns. If something has less than 2 types then it should be removed since it will not help classify

In [23]:
# Xx.groupby("steel").size() # stays
# Xx.groupby("product-type").size() # remove product-type
# Xx.groupby("carbon").size() # remove carbon as well since it has excessively 0s in it
# Xx.groupby("hardness").size()
# Xx.groupby("strength").size()
# Xx.groupby("shape").size() # stay
# Xx.groupby("thick").size() # continuous value stays
# Xx.groupby("width").size() # continuous value stays
# Xx.groupby("len").size() # continuous value stays
# Xx.groupby("bore").size() # remove
Xx.groupby("surface_quality").size() # needs pruning

surface_quality
?    217
D     50
E    278
F     53
G    199
dtype: int64

In [24]:
Xd = Xx[["steel", "shape", "thick", "width", "len", "surface_quality"]]
Xd

,steel,shape,thick,width,len,surface_quality
0,R,COIL,3.200,610.0,0,E
1,R,SHEET,0.700,1300.0,762,E
2,A,COIL,2.801,385.1,0,G
3,A,SHEET,0.801,255.0,269,G
4,A,COIL,1.600,610.0,0,D
...,...,...,...,...,...,...
891,R,SHEET,NaN,610.0,762,NaN
892,R,SHEET,NaN,830.0,880,NaN
893,V,SHEET,NaN,150.0,762,NaN
894,A,COIL,NaN,20.0,0,NaN


#### reindex so that encoder is applied easier this way

In [136]:
# Xd.isin(['?']).sum(axis=0)
xt.isin(['?']).sum(axis=0)

steel    0
shape    0
thick    0
width    0
len      0
dtype: int64

In [29]:
import numpy as np

In [37]:
Xd.isnull().sum()

surface_quality    99
steel               0
shape               0
thick              99
width               0
len                 0
dtype: int64

In [66]:
Xn = Xd.replace('?', np.nan)

In [67]:
Xn.isnull().sum(axis=0)

surface_quality    316
steel               86
shape                0
thick               99
width                0
len                  0
dtype: int64

In [41]:
Xd.shape

(896, 6)

In [68]:
Xn.groupby('surface_quality').size()

surface_quality
D     50
E    278
F     53
G    199
dtype: int64

#### Since most values in surface quality are np.nan so it will have to be dropped as well

In [69]:
Xn = Xn[["steel", "shape", "thick", "width", "len"]]
Xn

,steel,shape,thick,width,len
0,R,COIL,3.200,610.0,0
1,R,SHEET,0.700,1300.0,762
2,A,COIL,2.801,385.1,0
3,A,SHEET,0.801,255.0,269
4,A,COIL,1.600,610.0,0
...,...,...,...,...,...
891,R,SHEET,NaN,610.0,762
892,R,SHEET,NaN,830.0,880
893,V,SHEET,NaN,150.0,762
894,A,COIL,NaN,20.0,0


#### impute the missing values, the column "steel" has categorical data which I am going to replace for "?" with most frequent occourance

In [90]:
# from sklearn.impute import SimpleImputer as Imputer

In [61]:
# mf = imputer(missing_values="?", strategy='most_frequent') # from the other notebook
# mean = imputer(missing_values=np.nan, strategy='median')

In [81]:
xt = cp.copy(Xn)

In [105]:
# from sklearn.preprocessing import Imputer
# imputer = Imputer(missing_values = 'NaN', strategy = 'mean')
# xt = xt.replace('NaN', np.nan)
xt
# imputer = imputer.fit(xt[:,2])
# xt[:, 1:3] = imputer.transform(xt[:, 1:3])

,steel,shape,thick,width,len
0,R,COIL,3.200,610.0,0
1,R,SHEET,0.700,1300.0,762
2,A,COIL,2.801,385.1,0
3,A,SHEET,0.801,255.0,269
4,A,COIL,1.600,610.0,0
...,...,...,...,...,...
891,R,SHEET,NaN,610.0,762
892,R,SHEET,NaN,830.0,880
893,V,SHEET,NaN,150.0,762
894,A,COIL,NaN,20.0,0


In [133]:
xt.isnull().sum(axis=0)

steel    0
shape    0
thick    0
width    0
len      0
dtype: int64

In [108]:
xt = xt.dropna()
xt

,steel,shape,thick,width,len
0,R,COIL,3.200,610.0,0
1,R,SHEET,0.700,1300.0,762
2,A,COIL,2.801,385.1,0
3,A,SHEET,0.801,255.0,269
4,A,COIL,1.600,610.0,0
...,...,...,...,...,...
791,R,COIL,0.900,966.0,0
792,A,COIL,1.001,50.0,0
794,A,COIL,0.400,609.9,0
795,A,SHEET,3.200,610.0,4880


In [119]:
xt

,steel,shape,thick,width,len
0,R,COIL,3.200,610.0,0
1,R,SHEET,0.700,1300.0,762
2,A,COIL,2.801,385.1,0
3,A,SHEET,0.801,255.0,269
4,A,COIL,1.600,610.0,0
...,...,...,...,...,...
791,R,COIL,0.900,966.0,0
792,A,COIL,1.001,50.0,0
794,A,COIL,0.400,609.9,0
795,A,SHEET,3.200,610.0,4880


In [125]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(categories = 'auto')
x = onehotencoder.fit_transform(xt.iloc[:,[0,1]]).toarray()
x

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [127]:
x[:,:-3]

array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]])

In [132]:
x[:,3]

array([1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
       1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 0.